In [12]:
import pandas as pd
#read the file
print("Hello")
df = pd.read_csv("./data.csv", index_col="index")
print(df.head())


Hello
              host       time method                           url  response  \
index                                                                          
0      ***.novo.dk  805465029    GET                     /ksc.html       200   
1      ***.novo.dk  805465031    GET    /images/ksclogo-medium.gif       200   
2      ***.novo.dk  805465051    GET  /images/MOSAIC-logosmall.gif       200   
3      ***.novo.dk  805465053    GET     /images/USA-logosmall.gif       200   
4      ***.novo.dk  805465054    GET    /images/NASA-logosmall.gif       200   

       bytes  
index         
0       7067  
1       5866  
2        363  
3        234  
4        786  


In [13]:
# removing all not needed GET requests
remove_media = df[
    df.url.str.contains(".html", case=False) & df.url.str.contains(".htm", case=False)
    | df.url.str.endswith("/")
]


In [14]:
#grouping by host (client user)
grouped_by_host = remove_media.groupby(by=["host"])
print(len(grouped_by_host))
# more_than_one_site = grouped_by_host.filter(lambda x: len(x) == 1)
#removing session with only one action
more_than_one_site = grouped_by_host.filter(lambda x: len(x) > 1)


33805


In [15]:
grouped_more_action = more_than_one_site.groupby(by=["host"])
# print(len(grouped_more_action.groups))
# grouped_more_action.head()
divided_by_diff = grouped_more_action["time"].transform(lambda x: x.diff(), ).fillna(0)
more_than_one_site["diff"] = divided_by_diff


In [30]:
grouped_with_diff = more_than_one_site.groupby(by=["host"])
more_than_one_site["average"] = grouped_with_diff["diff"].transform(lambda x: x.expanding().mean()+x.expanding().max())
more_than_one_site.head(15)

,host,time,method,url,response,bytes,diff,average
index,,,,,,,,
0,***.novo.dk,805465029,GET,/ksc.html,200,7067,0.0,0.000000e+00
6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678,39.0,5.850000e+01
12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922,313.0,4.303333e+02
13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264,2486387.0,3.108072e+06
18,***.novo.dk,807951832,GET,/shuttle/countdown/,200,4673,64.0,2.983748e+06
23,***.novo.dk,807951938,GET,/shuttle/countdown/liftoff.html,200,4665,106.0,2.900872e+06
26,***.novo.dk,807952060,GET,/shuttle/countdown/lps/fr.html,200,1879,122.0,2.841677e+06
37,007.thegap.com,805065755,GET,/shuttle/countdown/,200,3998,0.0,0.000000e+00
41,007.thegap.com,805065868,GET,/shuttle/missions/sts-71/mission-sts-71.html,200,12722,113.0,1.695000e+02


In [17]:
# jesli odleglość pomiędzy ostatnim jest większa niż średnia zacznij nową sesje
# more_than_one_site['previous_average']=more_than_one_site['average'].shift(1)
# more_than_one_site["session"] = grouped_with_diff.apply(lambda x: print(x))

In [18]:
more_than_one_site.head()

,host,time,method,url,response,bytes,diff,average
index,,,,,,,,
0,***.novo.dk,805465029,GET,/ksc.html,200,7067,0.0,0.0
6,***.novo.dk,805465068,GET,/shuttle/missions/missions.html,200,8678,39.0,39.0
12,***.novo.dk,805465381,GET,/shuttle/resources/orbiters/columbia.html,200,6922,313.0,352.0
13,***.novo.dk,807951768,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11264,2486387.0,2486739.0
18,***.novo.dk,807951832,GET,/shuttle/countdown/,200,4673,64.0,2486803.0
